# W266 Final Project - Load_DataCleansing_TextProcessing

In [32]:
# Initial data load, summary, and high level sanity checks

# Import a bunch of libraries.
import os
import sys
import glob
import time
from IPython.core.display import clear_output
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.corpus import brown
nltk.download('brown')
from nltk.corpus import webtext
#import parseyface

#Prod 
#raw_reviews = pd.read_csv('RR_Pared_All_Active_2018-07-09.csv')
#Dev:
raw_reviews = pd.read_csv('RR_Pared_All_Active_Dev.csv')
# Errors:
#raw_reviews = pd.read_csv('Error_Records.csv')
#raw_reviews = pd.read_csv('Error_Records_2.csv')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mfrazzini/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to /home/mfrazzini/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [33]:
path =r'.' # use your path
allFiles = glob.glob(path + "/RR_Pared_All_Active_SBD_2018-07-09_*.csv")
review_sentences = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, index_col=None, header=0)
    list_.append(df)
review_sentences = pd.concat(list_)

print(review_sentences.shape)

#review_sentences = pd.read_csv("RR_Pared_All_Active_SBD_2018-07-09_0.csv")
#print(review_sentences)
#print(review_sentences.loc[0])

(1070247, 3)


In [34]:
#Remove html special characters that end up in the review text
from bs4 import BeautifulSoup 


Spell Check Procedures with Custom Dictionary Adds

In [35]:
brands = pd.read_csv('Brands_All_Active_2018-07-09.csv')
features = pd.read_csv('FeatureDump_All_Active_2018-07-09T16_31_17.csv')

import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(brown.words())

#Add custom dictionary brands
for index, row in brands.iterrows():
    parsed_words = words(row['BRAND'])
    for w in parsed_words:
        if len(w) >=3:
            WORDS[w] += 1

#Add custom dictionary features
for index, row in features.iterrows():
    feature_blurb = row['FEATURENAME']
    if type(feature_blurb) is str: 
        nopunc = ''
        nopunc = [char for char in feature_blurb if char not in string.punctuation]
        nopunc = ''.join(nopunc)
        parsed_words = words(nopunc)
        for w in parsed_words:
            if len(w) >=2:
                WORDS[w] += 1

#Add custom dictionary collections
WORDS["tls"] += 1

#Add custom dictionary travel words
WORDS["repacked"] += 1
WORDS["repack"] += 1
WORDS["carryon"] += 1
WORDS["toiletry"] += 1
WORDS["laptop"] += 1
WORDS["carousel"] += 1
WORDS["kindle"] += 1
WORDS["ipad"] += 1
WORDS["unpacked"] += 1
WORDS["unpack"] += 1
WORDS["mac"] += 1
WORDS["iphone"] += 1
WORDS["dvd"] += 1
WORDS["tsa"] += 1
WORDS["repacking"] += 1
WORDS["overpack"] += 1
WORDS["cram"] += 1

#Add custom dictionary Misc
WORDS["ebag"] += 1
WORDS["downside"] += 1
WORDS["andor"] += 1
WORDS["classy"] += 1
WORDS["online"] += 1
WORDS["bike"] += 1
WORDS["hassle"] += 1
WORDS["rips"] += 1
WORDS["undies"] += 1
WORDS["alot"] += 1
WORDS["wouldnt"] += 1
WORDS["pricey"] += 1
WORDS["im"] += 1
WORDS["picky"] += 1
WORDS["unusable"] += 1
WORDS["manageable"] += 1
WORDS["yrs"] += 1
WORDS["neater"] += 1
WORDS["plusses"] += 1
WORDS["alot"] += 1
WORDS["gripe"] += 1


def dict_check(word):
    if word.lower() in WORDS:
        return(word)
    else:
        return ""

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


In [36]:
# Test mod for initial dictionary check
print(dict_check("comfortable"))
print(dict_check("confortable"))
print(correction("confortable"))
print(dict_check("Baggalini"))
print(correction("unpacked"))

comfortable

comfortable

unpacked


In [16]:
import csv

#review_sentences = pd.read_csv("Review_sentences_Dev_Active_2018-07-09.csv")


def text_process(text):
    #Takes in a string of text, then performs the following:
    #1. Remove all html special characters, with regex to pull partially remove html special characters
    #2. Spelling correction
    #3. Remove all punctuation
    #4. Remove all stopwords
    #5. Stemming of words
    #6. Return the cleaned text as a list of words
    if type(text) is str:
        review = []
        nopunc = ''
        nopunc = [char for char in text if char not in string.punctuation]
        nopunc = ''.join(nopunc)
        for word in nopunc.split():
            #print(word)
            if len(word) <= int(45):
                total_words[0] += 1
                if word.lower() not in stopwords.words('english'):
                    #html_decoded = BeautifulSoup(word, "lxml")
                    #html_decoded_string = html_decoded.string
                    if re.search('\w(\d+)\w', word): #html_decoded_string):
                        word = word.replace("34","\"")
                        word = word.replace("39","\'")
                    if word == dict_check(word):
                        correct[0] += 1
                        spell_corrected = word
                    else:
                        if word in corrections:
                            spell_corrected = corrections[word]
                            corrections[word] = spell_corrected
                            corrections_count[spell_corrected + ':' + word] += 1
                            corrected[0] += 1
                        else:
                            spell_corrected = correction(word)
                            if word != spell_corrected:
                                corrections[word] = spell_corrected
                                corrections_count[spell_corrected + ':' + word] += 1
                                corrected[0] += 1  
                            else:
                                corrections[word] = word
                                corrected_default[0] += 1
                    #porter = PorterStemmer()
                    #stemmed = porter.stem(spell_corrected)
                    review.append(spell_corrected) 
                else:
                    review.append(word)
            else:
                review.append('')
        return ' '.join(review)
    else:
        return ''

total_review_sentences = [0]
total_words = [0]
correct = [0]
corrected = [0]
corrected_default = [0]
corrections = dict()
corrections_count = Counter()

processed_testimonial = str

#f = open('debug.txt','w')
    
for index, row in review_sentences.iterrows():
    total_review_sentences = index
    clear_output()
    print("On record: {}; ".format(index), "total words", total_words, "words correct: ", correct, \
          "corrected: ", corrected, "corrected default:", corrected_default, end="\r")
    review_sentences.at[index, 'SENTENCE'] = text_process(row['SENTENCE'])

print("Total review sentences: ", total_review_sentences)
print("Total words: ", total_words)  

review_sentences.to_csv("Review_sentences_Prod_Active_Cleansed_2018-07-09.csv")
    

Total review sentences:  132350 [13752263] words correct:  [6839826] corrected:  [216521] corrected default: [34302]
Total words:  [13752265]


In [49]:
review_sentences = pd.read_csv("Review_sentences_Prod_Active_Cleansed_2018-07-09.csv")
review_sentences['SENTENCE'].describe()

count     1069186
unique     123796
top       Love it
freq         3395
Name: SENTENCE, dtype: object

In [23]:
# print spell corrections for analysis and tuning
f = open("corrections_count.csv", encoding='utf-8-sig', mode='w')  
f .write('SPELL_CORRECTION, COUNT\n')  
for key, value in corrections_count.items():  
    f.write('{},{}\n'.format(key, value))
f.close()

In [25]:
corrections_count.most_common(100)

[("I'm:I'm", 11590),
 ('I:Im', 8392),
 ("I've:I've", 7610),
 ('ve:Ive', 5992),
 ('didn:didnt', 3443),
 ("I'll:I'll", 1988),
 ("I'd:I'd", 1916),
 ('haven:havent', 1900),
 ('slot:alot', 1609),
 ('want:wasnt', 1175),
 ('wellmade:wellmade', 1169),
 ('couldnt:wouldnt', 1055),
 ('price:pricey', 945),
 ('17:17"', 883),
 ('15:15"', 672),
 ('Haven:Havent', 635),
 ('21:21"', 530),
 ('a:¿', 483),
 ('25:25"', 477),
 ('in:im', 449),
 ('packing:repacking', 420),
 ('or:yr', 405),
 ('under:undies', 403),
 ('definitely:definately', 382),
 ('I:I¿m', 358),
 ('Spain:Spain', 340),
 ('France:France', 339),
 ('accommodate:accomodate', 321),
 ('bag:bag59', 317),
 ("bags:bag's", 309),
 ('welldesigned:welldesigned', 309),
 ('13:13"', 308),
 ('stuff:"stuff"', 295),
 ('baggallini:Baggalini', 291),
 ('pick:picky', 284),
 ('its:it¿s', 284),
 ('Its:It¿s', 281),
 ('June:June', 278),
 ('unable:unusable', 273),
 ('chin:chka', 272),
 ('Vegas:Vegas', 269),
 ('have:I¿ve', 267),
 ('unmanageable:manageable', 266),
 ('cream: